In [1]:
%conda install matplotlib -y
%pip install tpot
%conda install tpot -y

from scipy.stats import norm
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn import datasets

import numpy as np
import pandas as pd
import xgboost as xgb
import matplotlib.pyplot as plt
import seaborn as sns

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 23.3.1
  latest version: 23.10.0

Please update conda by running

    $ conda update -n base -c defaults conda

Or to minimize the number of packages updated during conda update use

     conda install conda=23.10.0



# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.

PackagesNotFoundError: The following packages are not available from current channels:

  - tpot

Current channels:

  - https://repo.anaconda.com/pkgs/main/osx-arm64
  - https://repo.anaconda.com/pkgs/main/noarch
  - https://repo.anaconda.com/pkgs/r/osx-arm64
  - https://repo.anaconda.com/pkgs/r/noarch

To search for alternate chann

# K-fold cross validation, Hyperparameter Tuning - practice
Goal: Create a model using KNN, but use grid search to determine hyperparameters and test with K-fold cross validation (K=10).


## Classifying Wine

The following is data containing information related to wine and whether it is drinkable.  
Predict the type ('target') using KNN and grid search, but achieve at least 99% accuracy (K-fold cross validation, based on K=10).

In [5]:
raw_wine = datasets.load_wine()
wine = pd.DataFrame(raw_wine['data'], index=raw_wine['target'], columns=raw_wine['feature_names'])
wine = wine.reset_index(names=['target'] + raw_wine['feature_names'])
display(wine)

,target,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline
0,0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0
1,0,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0
2,0,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0
3,0,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0
4,0,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,2,13.71,5.65,2.45,20.5,95.0,1.68,0.61,0.52,1.06,7.70,0.64,1.74,740.0
174,2,13.40,3.91,2.48,23.0,102.0,1.80,0.75,0.43,1.41,7.30,0.70,1.56,750.0
175,2,13.27,4.28,2.26,20.0,120.0,1.59,0.69,0.43,1.35,10.20,0.59,1.56,835.0
176,2,13.17,2.59,2.37,20.0,120.0,1.65,0.68,0.53,1.46,9.30,0.60,1.62,840.0


In [6]:
compact_columns_names =['target',
 'malic_acid',
 'ash',
 'alcalinity_of_ash',
 'magnesium',
 'nonflavanoid_phenols',
 'proanthocyanins',
 'color_intensity',
 'hue',
 'od280/od315_of_diluted_wines']

In [8]:
# Preprocessing 
X = wine[compact_columns_names]
y = wine["target"]

from sklearn.model_selection import train_test_split
train, test = train_test_split(wine, test_size=.2, random_state=2)
train_X = train[compact_columns_names]
train_Y = train["target"]

test_X = test[compact_columns_names]
test_Y = test["target"]
#Feature Scaling 
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X)

train_X = scaler.transform(train_X)
test_X = scaler.transform(test_X)

In [10]:
print(f"Training X: {train_X}\n Training Y: {train_Y}")

Training X: [[ 1.37386437 -0.93926832 -0.97420996 ...  1.09955402 -1.65599853
  -1.49956991]
 [ 0.07996036  0.05713777  0.19549913 ... -0.78212541 -0.69078402
   1.09932487]
 [ 0.07996036 -0.90336179 -0.24314178 ... -1.43530608  0.49379744
   0.84508517]
 ...
 [ 0.07996036 -0.60713296 -0.46246223 ... -0.57016612  0.09893695
   0.23773476]
 [-1.21394365 -0.47248348  1.21899459 ...  0.96978302  1.41513857
   0.37897904]
 [ 1.37386437  0.21871714  1.18244118 ...  1.5580782  -0.95402434
  -1.1464592 ]]
 Training Y: 154    2
120    1
89     1
92     1
156    2
      ..
43     0
22     0
72     1
15     0
168    2
Name: target, Length: 142, dtype: int64


In [11]:
# Preprocessing 
X = wine.iloc[:, :-1].values
y = wine.iloc[:, 0].values

#Feature Scaling 
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X)

X = scaler.transform(X)

In [12]:
### With K-fold cross validation

from sklearn.model_selection import StratifiedKFold
kfold = StratifiedKFold(n_splits = 10, shuffle = False)

accs = []

for fold, (train_idx, test_idx) in enumerate(kfold.split(X, y)):
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]

    # Training and Predictions
    from sklearn.neighbors import KNeighborsClassifier
    classifier = KNeighborsClassifier(n_neighbors=2)
    classifier.fit(X_train, y_train)

    y_pred = classifier.predict(X_test)

    # Evaluating the Algorithm
    from sklearn.metrics import confusion_matrix, accuracy_score
    print(confusion_matrix(y_test, y_pred))
    print(f"[Fold no.{fold}]: {accuracy_score(y_test, y_pred)*100:.3f}%")

    accs.append(accuracy_score(y_test, y_pred))

print(f"Average accuracy measured by K-fold cross validation: {sum(accs)/len(accs)*100:.3f}%")

[[6 0 0]
 [2 5 0]
 [0 1 4]]
[Fold no.0]: 83.333%
[[6 0 0]
 [1 6 0]
 [0 0 5]]
[Fold no.1]: 94.444%
[[6 0 0]
 [1 6 0]
 [0 0 5]]
[Fold no.2]: 94.444%
[[6 0 0]
 [0 7 0]
 [0 0 5]]
[Fold no.3]: 100.000%
[[6 0 0]
 [0 7 0]
 [0 0 5]]
[Fold no.4]: 100.000%
[[6 0 0]
 [0 7 0]
 [0 0 5]]
[Fold no.5]: 100.000%
[[6 0 0]
 [0 7 0]
 [0 0 5]]
[Fold no.6]: 100.000%
[[6 0 0]
 [0 7 0]
 [0 0 5]]
[Fold no.7]: 100.000%
[[6 0 0]
 [1 5 1]
 [0 0 4]]
[Fold no.8]: 88.235%
[[5 0 0]
 [1 7 0]
 [0 0 4]]
[Fold no.9]: 94.118%
Average accuracy measured by K-fold cross validation: 95.458%


In [14]:
# Preprocessing
X = wine[compact_columns_names]
y = wine["target"]

#Feature Scaling 
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X)

X = scaler.transform(X)

In [15]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier()

from sklearn.model_selection import StratifiedKFold
kfold = StratifiedKFold(n_splits = 10, shuffle = False)

grid_vals = {'n_neighbors': [1, 2, 3, 4,5,6,7,8,9,10,11,12,13,14]}
grid_knn = GridSearchCV(estimator=classifier, param_grid=grid_vals, scoring='accuracy', 
                       cv=kfold, refit=True, return_train_score=True) 

# Training and Prediction
grid_knn.fit(X, y)

# Evaluating the Algorithm
display(grid_knn.cv_results_)

{'mean_fit_time': array([0.00030878, 0.00022132, 0.00021102, 0.00021122, 0.00021029,
        0.00020738, 0.00021045, 0.00021644, 0.00020912, 0.00020838,
        0.00021126, 0.00022223, 0.00021265, 0.0002111 ]),
 'std_fit_time': array([1.77253798e-04, 1.65250220e-05, 5.00957094e-06, 3.11490019e-06,
        2.59646480e-06, 2.45420632e-06, 3.15407145e-06, 5.41876706e-06,
        2.90724061e-06, 1.98332160e-06, 3.15479226e-06, 1.58039150e-05,
        5.88121178e-06, 2.85744443e-06]),
 'mean_score_time': array([0.00058262, 0.00042343, 0.00041029, 0.00040727, 0.00041151,
        0.00041063, 0.00041547, 0.00042479, 0.00041692, 0.00041494,
        0.00042024, 0.00044274, 0.00043259, 0.00042584]),
 'std_score_time': array([2.79539639e-04, 2.32291162e-05, 6.93830837e-06, 5.24607565e-06,
        6.86711732e-06, 2.52554945e-06, 5.54157817e-06, 6.90348396e-06,
        4.07277277e-06, 5.89448652e-06, 6.05902250e-06, 2.15055673e-05,
        6.09176966e-06, 4.87033780e-06]),
 'param_n_neighbors': mask

In [17]:
# Best estimator
from sklearn.model_selection import StratifiedKFold
kfold = StratifiedKFold(n_splits = 10, shuffle = False)

accs = []

for fold, (train_idx, test_idx) in enumerate(kfold.split(X, y)):
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]

    y_pred = grid_knn.best_estimator_.predict(X_test)

    # Evaluating the Algorithm
    from sklearn.metrics import confusion_matrix, accuracy_score
    print(confusion_matrix(y_test, y_pred))
    print(f"[Fold no.{fold}]: {accuracy_score(y_test, y_pred)*100:.3f}%")

    accs.append(accuracy_score(y_test, y_pred))

print(f"Average accuracy measured by K-fold cross validation (best model): {sum(accs)/len(accs)*100:.3f}%")

[[6 0 0]
 [0 7 0]
 [0 0 5]]
[Fold no.0]: 100.000%
[[6 0 0]
 [0 7 0]
 [0 0 5]]
[Fold no.1]: 100.000%
[[6 0 0]
 [0 7 0]
 [0 0 5]]
[Fold no.2]: 100.000%
[[6 0 0]
 [0 7 0]
 [0 0 5]]
[Fold no.3]: 100.000%
[[6 0 0]
 [0 7 0]
 [0 0 5]]
[Fold no.4]: 100.000%
[[6 0 0]
 [0 7 0]
 [0 0 5]]
[Fold no.5]: 100.000%
[[6 0 0]
 [0 7 0]
 [0 0 5]]
[Fold no.6]: 100.000%
[[6 0 0]
 [0 7 0]
 [0 0 5]]
[Fold no.7]: 100.000%
[[6 0 0]
 [0 7 0]
 [0 0 4]]
[Fold no.8]: 100.000%
[[5 0 0]
 [0 8 0]
 [0 0 4]]
[Fold no.9]: 100.000%
Average accuracy measured by K-fold cross validation (best model): 100.000%
